In [ ]:
!pip install tensorflow-gpu==1.15.0

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras import layers
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import xml.etree.ElementTree as ET, getopt, logging, sys, random, re, copy, os
from lxml import etree
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

from keras import backend as K

Using TensorFlow backend.


In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/SVM/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

In [ ]:
def getSentences(file):
  tree = ET.parse(file, etree.XMLParser(recover=True, encoding="utf-8"))
  root = tree.getroot()
  s = []
  p = []
  #for review in root.findall('Review'):
  for sentence in root.findall('sentence'):
    #for sentence in sentences.findall('sentence'):
    sent = []
    sent_characteristics = []
    text = sentence.find('text').text
    sent.append(text)
    polarity = []
    for opinions in sentence.findall('aspectCategories'):
      for opinion in opinions.findall('aspectCategory'):
        elem = [opinion.get('category'), opinion.get('polarity')]
        polarity.append(elem)
    sent_characteristics.append(polarity)
    s.append(sent)
    p.append(sent_characteristics)
        
  return s, p

In [ ]:
train_sentences, train_adnotations = getSentences("./train2014.xml")
test_sentences, test_adnotations = getSentences("./test2014.xml")

In [ ]:
test_adnotations

[[[['food', None]]],
 [[['service', None]]],
 [[['food', None]]],
 [[['food', None]]],
 [[['food', None], ['ambience', None]]],
 [[['anecdotes/miscellaneous', None]]],
 [[['food', None]]],
 [[['food', None]]],
 [[['food', None]]],
 [[['food', None]]],
 [[['food', None]]],
 [[['food', None], ['service', None]]],
 [[['food', None], ['ambience', None]]],
 [[['service', None]]],
 [[['food', None]]],
 [[['food', None]]],
 [[['food', None], ['price', None]]],
 [[['food', None], ['service', None]]],
 [[['food', None]]],
 [[['service', None]]],
 [[['service', None], ['anecdotes/miscellaneous', None]]],
 [[['food', None], ['anecdotes/miscellaneous', None]]],
 [[['food', None]]],
 [[['anecdotes/miscellaneous', None]]],
 [[['anecdotes/miscellaneous', None]]],
 [[['food', None]]],
 [[['anecdotes/miscellaneous', None]]],
 [[['anecdotes/miscellaneous', None]]],
 [[['food', None]]],
 [[['food', None]]],
 [[['food', None], ['service', None]]],
 [[['food', None], ['ambience', None]]],
 [[['food', None]

In [ ]:
train_reviews = []
train_aspects = []
test_reviews = []
test_aspects = []

In [ ]:
for review in train_sentences:
  train_reviews.append(' '.join(review))
for ta in train_adnotations:
  aspect = set()
  for adnotation_set in ta:
    for a in adnotation_set:
      aspect.add(a[0])
  train_aspects.append(aspect)

In [ ]:
for review in test_sentences:
  test_reviews.append(' '.join(review))
  
for ta in test_adnotations:
  aspect = set()
  for adnotation_set in ta:
    for a in adnotation_set:
      aspect.add(a[0])
  test_aspects.append(aspect)

In [ ]:
def getLabels(aspects):
	#print(unique_aspects)
	#Create train labels
  food = []
  service	= []
  miscellaneous = []
  ambience = []
  price = []

  for aspect in aspects:
    if 'food' in aspect:
      food.append(1)
    else:
      food.append(0)
			
    if 'anecdotes/miscellaneous' in aspect:
      miscellaneous.append(1)
    else:
      miscellaneous.append(0)
			
    if 'service' in aspect:
      service.append(1)
    else:
      service.append(0)

    if 'ambience' in aspect:
      ambience.append(1)
    else:
      ambience.append(0)
			  
    if 'price' in aspect:
      price.append(1)
    else:
      price.append(0)
				
  return food, miscellaneous ,service ,price , ambience

In [ ]:
#Test and Train labels
test1, test2, test3, test4, test5 = getLabels(test_aspects)
test_labels = [test1, test2, test3, test4, test5]

train1, train2, train3, train4, train5 = getLabels(train_aspects)
train_labels = [train1, train2, train3, train4, train5]

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [ ]:
!pip install bert-tensorflow

In [ ]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [ ]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

In [ ]:
train = pd.DataFrame(list(zip(train_reviews, train5)), 
               columns =['sentence', 'polarity']) 
test = pd.DataFrame(list(zip(test_reviews, test5)), 
               columns =['sentence', 'polarity']) 

In [ ]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [ ]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 3041


INFO:tensorflow:Writing example 0 of 3041


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


In [ ]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [ ]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [ ]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [ ]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [ ]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpjzgeq_ni', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdfb6a715c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpjzgeq_ni', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdfb6a715c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [ ]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 285 into /tmp/tmpjzgeq_ni/model.ckpt.


INFO:tensorflow:Saving checkpoints for 285 into /tmp/tmpjzgeq_ni/model.ckpt.


INFO:tensorflow:Loss for final step: 0.003011277.


INFO:tensorflow:Loss for final step: 0.003011277.


Training took time  0:08:46.167680


In [ ]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
estimator.evaluate(input_fn=test_input_fn, steps=None)


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 285: auc = 0.9177022, eval_accuracy = 0.96125, f1_score = 0.86695266, false_negatives = 17.0, false_positives = 14.0, global_step = 285, loss = 0.15127382, precision = 0.87826085, recall = 0.8559322, true_negatives = 668.0, true_positives = 101.0


INFO:tensorflow:Saving dict for global step 285: auc = 0.9177022, eval_accuracy = 0.96125, f1_score = 0.86695266, false_negatives = 17.0, false_positives = 14.0, global_step = 285, loss = 0.15127382, precision = 0.87826085, recall = 0.8559322, true_negatives = 668.0, true_positives = 101.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 285: /tmp/tmpjzgeq_ni/model.ckpt-285


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 285: /tmp/tmpjzgeq_ni/model.ckpt-285


{'auc': 0.9177022,
 'eval_accuracy': 0.96125,
 'f1_score': 0.86695266,
 'false_negatives': 17.0,
 'false_positives': 14.0,
 'global_step': 285,
 'loss': 0.15127382,
 'precision': 0.87826085,
 'recall': 0.8559322,
 'true_negatives': 668.0,
 'true_positives': 101.0}

**END OF BERT**

In [ ]:
xx

NameError: ignored

In [ ]:
#Vectorizing data
vectorizer = CountVectorizer(analyzer='word', lowercase=True, stop_words='english', ngram_range=(1,2))
vectorizer.fit(train_reviews)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
x_train = vectorizer.transform(train_reviews)
x_test = vectorizer.transform(test_reviews)

In [ ]:
x_train = x_train.toarray()

In [ ]:
x_train[0:2]

In [ ]:
print(x_train[1:2])

In [ ]:
input_dim = x_train.shape[1]

tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_reviews)

In [ ]:
x_train = tokenizer.texts_to_sequences(train_reviews)
x_test = tokenizer.texts_to_sequences(test_reviews)
vocab_size = len(tokenizer.word_index) + 1 

In [ ]:
print(x_train[0:2])

In [ ]:
maxlen = 300
x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding = 'post', maxlen=maxlen)

In [ ]:
x_train[0:1]

In [ ]:
#Pretrained Word Embeddings
def create_embedding_matrix(filepath, word_index, embedding_dim):
	vocab_size = len(word_index) + 1
	embedding_matrix = np.zeros((vocab_size, embedding_dim))
	
	with open(filepath, encoding='utf-8') as f:
		for line in f:
			word, *vector = line.split()
			if word in word_index:
				idx = word_index[word]
				embedding_matrix[idx] = np.array(vector[-300:], dtype=np.float32)[:embedding_dim]

	return embedding_matrix

In [ ]:
embedding_dim = 300
embedding_matrix = create_embedding_matrix('./glove.840B.300d.txt', tokenizer.word_index, embedding_dim)

In [ ]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis = 1))

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping
def getPredictions(x_train, x_test, train, test):
	embedding_dim = 300
	#early_stopping_monitor = EarlyStopping(patience=3)
	embedding_matrix = create_embedding_matrix('./glove.840B.300d.txt', tokenizer.word_index, embedding_dim)
	model = Sequential()
	model.add(layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length = maxlen, trainable = True))
	model.add(layers.Conv1D(64, 3, activation = 'relu'))
	model.add(layers.GlobalMaxPool1D())
	model.add(layers.Dense(10, activation='relu'))
	model.add(layers.Dense(1, activation='sigmoid'))
	model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1, 'accuracy'])
	#model.summary()

	history = model.fit(x_train, train, epochs = 20, verbose = 1, validation_data = (x_test, test), batch_size = 10)
	val = model.evaluate(x_train, train, verbose = False)
	#val = model.evaluate(x_test, test, verbose = False)

	predictions = model.predict(x_test)
	predictions_class = model.predict_classes(x_test)
	#predictions1 = model.predict(testt)
	#predictions11 = model.predict_classes(testt)
	return predictions, predictions_class

In [ ]:
embedding_dim = 300
from gensim.models import KeyedVectors
word2vec_model = KeyedVectors.load_word2vec_format(os.path.join('./GoogleNews-vectors-negative300.bin.gz'), binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model: 
        embedding_vector = word2vec_model[word]
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=True)

In [ ]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping
def getPredictions(x_train, x_test, train, test):
	embedding_dim = 300
	#early_stopping_monitor = EarlyStopping(patience=3)
	#embedding_matrix = create_embedding_matrix('./glove.840B.300d.txt', tokenizer.word_index, embedding_dim)
	model = Sequential()
	model.add(embedding_layer)
	model.add(layers.Conv1D(64, 3, activation = 'relu'))
	model.add(layers.GlobalMaxPool1D())
	model.add(layers.Dense(10, activation='relu'))
	model.add(layers.Dense(1, activation='sigmoid'))
	model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1, 'accuracy'])
	#model.summary()

	history = model.fit(x_train, train, epochs = 20, verbose = 1, validation_data = (x_test, test), batch_size = 10)
	val = model.evaluate(x_train, train, verbose = False)
	#val = model.evaluate(x_test, test, verbose = False)

	predictions = model.predict(x_test)
	predictions_class = model.predict_classes(x_test)
	#predictions1 = model.predict(testt)
	#predictions11 = model.predict_classes(testt)
	return predictions, predictions_class

In [ ]:
print("Getting Predictions1")
predictions1, predictions_class1 = getPredictions(x_train, x_test, train1, test1)

Getting Predictions1
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 3041 samples, validate on 800 samples
Epoch 1/20
3041/3041 [==============================] - 9s 3ms/step - loss: 0.3157 - f1: 0.8228 - accuracy: 0.8800 - val_loss: 0.2259 - val_f1: 0.9072 - val_accuracy: 0.9187
Epoch 2/20
3041/3041 [==============================] - 3s 891us/step - loss: 0.1309 - f1: 0.9224 - accuracy: 0.9494 - val_loss: 0.2132 - val_f1: 0.9036 - val_accuracy: 0.9162
Epoch 3/20
3041/3041 [==============================] - 3s 880us/step - loss: 0.0493 - f1: 0.9688 - accuracy: 0.9865 - val_loss: 0.2591 - val_f1: 0.8947 - val_accuracy: 0.9100
Epoch 4/20
3041/3041 [==============================] - 3s 883us/step - loss: 0.0151 - f1: 0.9958 - accuracy: 0.9974 - val_loss: 0.3142 - val_f1: 0.8999 - val_accuracy: 0.9137
Epoch 5/20
3041/3041 [==============================] 

In [ ]:
print("Getting Predictions2")
predictions2, predictions_class2 = getPredictions(x_train, x_test, train2, test2)
print("Getting Predictions3")
predictions3, predictions_class3 = getPredictions(x_train, x_test, train3, test3)
print("Getting Predictions4")
predictions4, predictions_class4 = getPredictions(x_train, x_test, train4, test4)
print("Getting Predictions5")
predictions5, predictions_class5 = getPredictions(x_train, x_test, train5, test5)

Getting Predictions2
Train on 3041 samples, validate on 800 samples
Epoch 1/20
3041/3041 [==============================] - 3s 955us/step - loss: 0.3710 - f1: 0.7311 - accuracy: 0.8356 - val_loss: 0.3526 - val_f1: 0.6671 - val_accuracy: 0.8462
Epoch 2/20
3041/3041 [==============================] - 3s 886us/step - loss: 0.1636 - f1: 0.8903 - accuracy: 0.9398 - val_loss: 0.3692 - val_f1: 0.6686 - val_accuracy: 0.8625
Epoch 3/20
3041/3041 [==============================] - 3s 925us/step - loss: 0.0483 - f1: 0.9654 - accuracy: 0.9872 - val_loss: 0.4488 - val_f1: 0.6571 - val_accuracy: 0.8512
Epoch 4/20
3041/3041 [==============================] - 3s 871us/step - loss: 0.0120 - f1: 0.9862 - accuracy: 0.9993 - val_loss: 0.5267 - val_f1: 0.6604 - val_accuracy: 0.8537
Epoch 5/20
3041/3041 [==============================] - 3s 886us/step - loss: 0.0037 - f1: 0.9869 - accuracy: 1.0000 - val_loss: 0.5754 - val_f1: 0.6635 - val_accuracy: 0.8550
Epoch 6/20
3041/3041 [==============================

In [ ]:
predictions1 = predictions1[:, 0]
predictions_class1 = predictions_class1[:, 0]

predictions2 = predictions2[:, 0]
predictions_class2 = predictions_class2[:, 0]

predictions3 = predictions3[:, 0]
predictions_class3 = predictions_class3[:, 0]

predictions4 = predictions4[:, 0]
predictions_class4 = predictions_class4[:, 0]

predictions5 = predictions5[:, 0]
predictions_class5 = predictions_class5[:, 0]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
print("Accuracy1: ", accuracy1)
print("Precision1",precision1)
print("Recall1",recall1)
print("Auc1",auc1)
print("F1_score1",f1_score1)

Accuracy1:  0.9175
Precision1 0.9559585492227979
Recall1 0.8827751196172249
Auc1 0.9191362509080888
F1_score1 0.9179104477611939


In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy1 = accuracy_score(test1, predictions_class1)
accuracy2 = accuracy_score(test2, predictions_class2)
accuracy3 = accuracy_score(test3, predictions_class3)
accuracy4 = accuracy_score(test4, predictions_class4)
accuracy5 = accuracy_score(test5, predictions_class5)
final_accuracy = (accuracy1 + accuracy2 + accuracy3 + accuracy4 + accuracy5) / 5
final_accuracy

0.92875

In [ ]:
# precision tp / (tp + fp)
precision1 = precision_score(test1, predictions_class1)
precision2 = precision_score(test2, predictions_class2)
precision3 = precision_score(test3, predictions_class3)
precision4 = precision_score(test4, predictions_class4)
precision5 = precision_score(test5, predictions_class5)
final_precision_score = (precision1 + precision2 + precision3 + precision4 + precision5) / 5
final_precision_score

0.8993400352352132

In [ ]:
# recall: tp / (tp + fn)
recall1 = recall_score(test1, predictions_class1)
recall2 = recall_score(test2, predictions_class2)
recall3 = recall_score(test3, predictions_class3)
recall4 = recall_score(test4, predictions_class4)
recall5 = recall_score(test5, predictions_class5)
final_recall_score = (recall1 + recall2 + recall3 + recall4 + recall5) / 5
final_recall_score 

0.7923315874204924

In [ ]:
# f1: 2 tp / (2 tp + fp + fn)
f1_score1 = f1_score(test1, predictions_class1)
f1_score2 = f1_score(test2, predictions_class2)
f1_score3 = f1_score(test3, predictions_class3)
f1_score4 = f1_score(test4, predictions_class4)
f1_score5 = f1_score(test5, predictions_class5)
final_f1_score = (f1_score1 + f1_score2 + f1_score3 + f1_score4 + f1_score5) / 5
final_f1_score  

0.841148579311892

In [ ]:
f1_score1

0.9179104477611939

In [ ]:
# ROC AUC
auc1 = roc_auc_score(test1, predictions_class1)
auc2 = roc_auc_score(test2, predictions_class2)
auc3 = roc_auc_score(test3, predictions_class3)
auc4 = roc_auc_score(test4, predictions_class4)
auc5 = roc_auc_score(test5, predictions_class5)
final_auc = (auc1 + auc2 + auc3 + auc4 + auc5) / 5
final_auc

0.878677898926506

In [ ]:
predicted_aspects = []
nr = len(test_sentences)

for i in range(nr):
  predicted_aspect = []
  
  if predictions1[i] > 0.4:
    predicted_aspect.append('food')
  if predictions2[i] > 0.4:
    predicted_aspect.append('miscellaneous')
  if predictions3[i] > 0.4:
    predicted_aspect.append('service')
  if predictions4[i] > 0.4:
    predicted_aspect.append('price')
  if predictions5[i] > 0.4:
    predicted_aspect.append('ambience')
    
  predicted_aspects.append(predicted_aspect)

In [ ]:
#Evaluating the system
common_aspects = 0
relevant_aspects = 0
retrieved_aspects = 0

for i in range(nr):
  correct = set()
  for aspect in test_aspects[i]:
    correct.add(aspect)
  
  predicted = set()
  for aspect in predicted_aspects[i]:
    predicted.add(aspect)
    
  relevant_aspects = relevant_aspects + len(correct)
  retrieved_aspects = retrieved_aspects + len(predicted)
  common_aspects = common_aspects+len([aspect for aspect in predicted if aspect in correct])

In [ ]:
print("Common aspects: ", common_aspects)
print("Retrieved aspects: ", retrieved_aspects)
print("Relevant aspects: ", relevant_aspects)
precision = common_aspects / retrieved_aspects if retrieved_aspects > 0 else 0
recall = common_aspects / relevant_aspects
f1_measure = 2 * precision * recall / (precision + recall)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1_measure)

Common aspects:  668
Retrieved aspects:  944
Relevant aspects:  1025
Precision:  0.7076271186440678
Recall:  0.6517073170731708
F1:  0.6785170137125445
